In [1]:
from match_audit import main
import pandas as pd

# Part 1 : Atoms without Episodes

``cat "C:\Users\aftab.jalal\dev\atom-analysis\data\out\audits\atoms__noep_for_slk_20230701-20231230.csv" | cut -f 1 -d ',' | uniq  > atoms_noep_for_slk.csv``

`grep -f atoms_noep_for_slk.csv "C:\Users\aftab.jalal\dev\atom-analysis\data\in\NSW_CSV\DATS_AllPrograms_01072023-30122023.csv"`

In [2]:
matched_eps, matched_atoms,  audit_results = main()
if not matched_eps:
  exit(1)

<class 'pandas.core.series.Series'>
RangeIndex: 739 entries, 0 to 738
Series name: CommencementDate
Non-Null Count  Dtype         
--------------  -----         
739 non-null    datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 5.9 KB


c:\Users\aftab.jalal\dev\atom-analysis\match_audit.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atoms__noep_for_slk.sort_values(by=col_str, ascending=True, inplace=True)
c:\Users\aftab.jalal\dev\atom-analysis\match_audit.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps__noatom_for_slk.sort_values(by=col_str, ascending=True, inplace=True)
c:\Users\aftab.jalal\dev\atom-analysis\utils\df_ops_base.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
list(unmatched_atoms_df.columns),  list(ep_df.columns)

(['SLK', 'Program', 'Staff', 'AssessmentDate', 'PDC'],
 ['SLK', 'Program', 'CommencementDate', 'EndDate', 'PDC'])

In [13]:
audit_results

{'NoEpisodeForSLK':                  SLK    Program             Staff AssessmentDate   
 959   ABCDT210719812  BegaMHHub       Aftab.Jalal     2023-10-23  \
 960   ACAIT230419891        TSS  Michelle.Figares     2023-09-21   
 961   ACHIC010419721       COCO     Mysh.Kirkwood     2023-08-01   
 962   ACIUS190519612        TSS        Mimi.Avila     2023-08-07   
 963   ACYON080819841        TSS  Michelle.Figares     2023-07-17   
 ...              ...        ...               ...            ...   
 1491  YPEAS271119761        TSS        Mimi.Avila     2023-09-19   
 1492  YREAR060119931       COCO     Mysh.Kirkwood     2023-12-08   
 1495  YSOOH171119681        TSS        Mimi.Avila     2023-09-13   
 1496  ZZPOE220719671   EUROPATH        Paul.Crowe     2023-11-09   
 1497  ZZRTE100819791        TSS      Nurgul.Sawut     2023-07-17   
 
                           PDC  
 959   Benzodiazepines, n.f.d.  
 960                    Heroin  
 961                   Ethanol  
 962               

## Summary

In [14]:
epcounts = ep_df.groupby(pd.Grouper(key='CommencementDate', freq='M')).count()
unmatched_atom_counts = unmatched_atoms_df.groupby(pd.Grouper(key='AssessmentDate', freq='M')).count()

In [15]:
epcounts[epcounts['SLK'] > 10]

,SLK,Program,EndDate,PDC
CommencementDate,,,,
2022-11-30,12,12,8,12
2022-12-31,11,11,5,11
2023-01-31,12,12,7,12
2023-02-28,16,16,10,16
2023-03-31,40,40,26,40
2023-04-30,22,22,12,22
2023-05-31,61,61,43,61
2023-06-30,69,69,48,69
2023-07-31,72,72,44,72


In [16]:
unmatched_atom_counts[unmatched_atom_counts['SLK'] > 10]

,SLK,Program,Staff,PDC
AssessmentDate,,,,
2023-07-31,117,117,117,117
2023-08-31,96,96,96,96
2023-09-30,115,115,115,115
2023-10-31,78,78,78,78
2023-11-30,104,104,104,104
2023-12-31,29,29,29,29


In [17]:
# atom_columns = ['SLK', 'Program', 'Staff_y','AssessmentDate_y']
# ep_columns=['SLK', 'Program', 'CommencementDate', 'EndDate']
slk = 'RIENT290619861'


In [18]:
ep_df.loc[ep_df['SLK'] == slk]


,SLK,Program,CommencementDate,EndDate,PDC


In [31]:
# unm_atom_slks = unmatched_atoms['SLK'].unique()
# ep_slks = ep_df['SLK'].unique()
# unm_atom_slks.loc[~unm_atom_slks.isin(ep_slks),'SLK']
atoms__noep_for_slk = unmatched_atoms.loc[~unmatched_atoms['SLK'].isin(ep_df['SLK'])]

In [32]:
atoms__noep_for_slk.to_csv('atoms__noep_for_slk.csv', index=False)

In [ ]:
# unmatched_atoms.loc[unmatched_atoms['SLK'] == slk,atom_columns]
# atoms__noep_for_slk['Staff'].value_counts()

In [12]:
# olli = atoms__noep_for_slk[atoms__noep_for_slk['Staff']=='Oliver.Lewis']

In [35]:
# Merge on 'SLK'
merged_df = pd.merge(unmatched_w_slk_match, ep_df, on='SLK', suffixes=('_atom', '_ep'))

# Find rows where 'Program' values don't match
mismatch_df = merged_df[merged_df['Program_atom'] != merged_df['Program_ep']]

In [38]:
# mismatch_df['Staff'].value_counts()

In [36]:
program_mismatch_for_matched_slk = mismatch_df[['SLK','Program_atom',	 'Program_ep','Staff',	'AssessmentDate','CommencementDate', 'EndDate']].drop_duplicates()

In [37]:
program_mismatch_for_matched_slk.to_csv('program_mismatch_for_matched_slk.csv', index=False)

In [18]:
# result = olli.groupby(pd.Grouper(key='AssessmentDate', freq='M')).count()


# Episodes without ATOMs